In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt

import torch.optim as optim
import torch.nn as nn

from qibo import Circuit, gates, hamiltonians, set_backend, construct_backend

from qiboml.models.encoding import PhaseEncoding
from qiboml.models.decoding import Expectation
from qiboml.interfaces.pytorch import QuantumModel
from qiboml.operations.differentiation import PSR, Jax

2025-03-27 10:34:36.583550: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743068076.602527  150751 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743068076.608310  150751 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
set_backend("qiboml", platform="pytorch")

nqubits = 1
nlayers = 3

# Encoding layer
encoding_circ = PhaseEncoding(
    nqubits=nqubits, 
    encoding_gate=gates.RX
)

# Trainable layer
def trainable_circuit(entanglement=True):
    trainable_circ  = Circuit(nqubits)
    for q in range(nqubits):
        trainable_circ.add(gates.RY(q=q, theta=np.random.randn()))
        trainable_circ.add(gates.RZ(q=q, theta=np.random.randn()))
    if nqubits > 1 and entanglement:
        [trainable_circ.add(gates.CNOT(q%nqubits, (q+1)%nqubits) for q in range(nqubits))]
    return trainable_circ

[Qibo 0.2.17|INFO|2025-03-27 10:34:45]: Using qiboml (pytorch) backend on cpu


In [3]:
def compute_gradient_wrt_x(
    execution_backend,
    differentiation_rule,
    trainable_circ_list,
):
    # Decoding layer
    decoding_circ = Expectation(
        nqubits=nqubits, 
        backend=execution_backend
    )

    structure = []
    for i, circ in enumerate(trainable_circ_list):
        structure.extend([
            encoding_circ, circ
        ])

    # The whole model
    model = QuantumModel(
        circuit_structure=structure,
        decoding=decoding_circ,
        differentiation=differentiation_rule,
    )

    data = torch.tensor([0.3], requires_grad=True)
    
    output = model(data)
    loss = output
    loss.backward()

    print(f"Grad via {differentiation_rule} on {execution_backend} backend:", data.grad)

In [4]:
trainable_circ_list = []
for _ in range(nlayers):
    trainable_circ_list.append(trainable_circuit(entanglement=True))

In [5]:
compute_gradient_wrt_x(
    execution_backend=construct_backend("numpy"),
    differentiation_rule=PSR(),
    trainable_circ_list=trainable_circ_list,
)
compute_gradient_wrt_x(
    execution_backend=construct_backend("qiboml", platform="pytorch"),
    differentiation_rule=None,
    trainable_circ_list=trainable_circ_list,
)
compute_gradient_wrt_x(
    execution_backend=construct_backend("numpy"),
    differentiation_rule=Jax(),
    trainable_circ_list=trainable_circ_list,
)

[array([[2.02280271]]), array([[-0.76928721]]), array([[0.55199038]]), array([[-0.13304896]]), array([[0.48786879]]), array([[-0.73194227]]), array([[-9.23266963e-10]])]
Grad via PSR() on numpy backend: tensor([2.0228])
Grad via None on qiboml (pytorch) backend: tensor([2.0228])
Grad via Jax() on numpy backend: tensor([2.0228])
